<a href="https://colab.research.google.com/github/Chaffre/Bio-Inspired-ER/blob/main/Mountain_car_Thomas_Chaffre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 723.9/723.9 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 34.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.1+cu116
    Uninstalling torchvision-0.14.1+cu116:
      Successfully uninstalled torchvision-0.14.1+cu116
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.14.1 requires torch==1.13.1, but you have torch 1.4.0+cu100 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==

In [2]:
pip install gym[classic_control]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 56.3 MB/s eta 0:00:00


In [3]:
import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))

pygame 2.1.0 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


<Surface(640x480x32 SW)>

In [4]:
import gym
import time
import matplotlib.pyplot as plt 
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as f

# Use Cuda GPU, if not available CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device used: " + str(device))

Device used: cpu


In [7]:
class Critic(nn.Module):

    def __init__(self,state_dim,act_dim):
        super(Critic,self).__init__()
        
        self.fc1=nn.Linear(11,16)
        self.fc2=nn.Linear(16,16)
        self.fc3=nn.Linear(16,1)

        self.fc1.weight.data.normal_(mean=0.0, std=1.0/(np.sqrt(0.5*(11))))
        self.fc1.bias.data.fill_(0.1)
        
        self.fc2.weight.data.normal_(mean=0.0, std=1.0/(np.sqrt(0.5*16)))
        self.fc2.bias.data.fill_(0.1)
        
        self.fc3.weight.data.normal_(mean=0.0, std=1.0/(np.sqrt(0.5*16)))
        self.fc3.bias.data.fill_(0.1)
        
        self.Ln_1 = nn.LayerNorm(16)
        self.Ln_2 = nn.LayerNorm(16)
        
    def forward(self,state,action):
        
        state = torch.reshape(state, (1, 10))
        action = torch.reshape(action, (1, 1))

        x = torch.cat([state, action], 1)
        
        x = f.leaky_relu(self.Ln_1(self.fc1(x)))

        x = f.leaky_relu(self.Ln_2(self.fc2(x)))

        x = self.fc3(x)
        
        return x

In [8]:
class Actor(nn.Module):

    def __init__(self,state_dim,act_dim):
        super(Actor,self).__init__()
        
        self.fc1=nn.Linear(10,16)
        self.fc2=nn.Linear(16,16)
        self.fc4=nn.Linear(16,16)
        
        self.fc3=nn.Linear(16,3)

        self.fc1.weight.data.normal_(mean=0.0, std=1.0/(np.sqrt(0.5*(10))))
        self.fc1.bias.data.fill_(0.1)
        
        self.fc2.weight.data.normal_(mean=0.0, std=1.0/(np.sqrt(0.5*16)))
        self.fc2.bias.data.fill_(0.1)

        self.fc4.weight.data.normal_(mean=0.0, std=1.0/(np.sqrt(0.5*16)))
        self.fc4.bias.data.fill_(0.1)
        
        self.fc3.weight.data.normal_(mean=0.0, std=1.0/(np.sqrt(0.5*16)))
        self.fc3.bias.data.fill_(0.1)
        
        self.Ln_1 = nn.LayerNorm(16)
        self.Ln_2 = nn.LayerNorm(16)
        self.Ln_4 = nn.LayerNorm(16)


    def forward(self,x):

      x = f.leaky_relu(self.Ln_1(self.fc1(x)))
        
      x = f.leaky_relu(self.Ln_2(self.fc2(x)))

      #x = f.relu(self.Ln_4(self.fc4(x)))

      x = self.fc3(x)
        
      return torch.tanh(x)

    def noise(self, noise_std):
      
      self.fc1.weight.add_(torch.randn(2).to(device) * noise_std)
      self.fc1.bias.add_(torch.randn(16).to(device) * noise_std)
      
      self.fc2.weight.add_(torch.randn(16).to(device) * noise_std)
      self.fc2.bias.add_(torch.randn(16).to(device) * noise_std)
      
      self.fc3.weight.add_(torch.randn(16).to(device) * noise_std)
      self.fc3.bias.add_(torch.randn(3).to(device) * noise_std)

In [9]:
class Bio_inspired_Buffer:
   
   def __init__(self, capacity):
      self.capacity = capacity
      self.buffers = [[], []]
      self.position = [0, 0]
      
   def push(self, state, action, reward, next_state, done, buffer_nb):
      
      if len(self.buffers[buffer_nb]) < self.capacity[buffer_nb]:
         self.buffers[buffer_nb].append(None)

      self.buffers[buffer_nb][self.position[buffer_nb]] = (state, action, reward, next_state, done)
      self.position[buffer_nb] = (self.position[buffer_nb] + 1) % self.capacity[buffer_nb]
   
   def CER(self, batch_size):
      
      batch = random.sample(self.buffers[0], batch_size)
      
      #batch.append(self.buffers[0][-1]) # CER Technique, source : https://arxiv.org/abs/1712.01275

      state, action, reward, next_state, done = map(np.stack, zip(*batch))
      
      return state, action, reward, next_state, done

In [18]:
def soft_q_update(batch_size, update_ratio, update,
              gamma = 0.99,
              mean_lambda = 1e-3,
              std_lambda = 1e-3,
              z_lambda = 0.0,
              soft_tau = 5e-3,
              total_q_value_loss = [],
              total_q_value_loss_2 = [],
              total_value_loss = [],
              total_policy_loss = [],
              total_alpha_loss = [],
              ):

   state, action, reward, next_state, done = replay_buffer.CER(batch_size)
   '''
   state = [x+np.random.normal(0.0, 0.2) for x in state]
   next_state = [x+np.random.normal(0.0, 0.2) for x in next_state]
   action = [x+np.random.normal(0.0, 0.2) for x in action]
   '''
   state      = torch.FloatTensor(state).to(device)
   next_state = torch.FloatTensor(next_state).to(device)
   action     = torch.FloatTensor(action).to(device)
   reward     = torch.FloatTensor(reward).unsqueeze(1).to(device)
   done       = torch.FloatTensor(np.float32(done)).unsqueeze(1).to(device)
   
   expected_q_value = soft_q_net(state, action)
   expected_q_value_2 = soft_q_net_2(state, action)

   #new_action = policy_net.forward(next_state)

   new_action = policy_net.forward(next_state)

   for i in range(len(new_action)):
     
     #if np.random.uniform(0.0, 1.0) < 0.50:

       #new_action[i] = np.random.randint(3)
     
     #else:
       
     new_action[i] = torch.nn.functional.softmax(new_action[i]) #+np.random.normal(0.0, 0.1))

   n_a = []
   for i in range(len(new_action)):
      
      n_a.append((torch.argmax(new_action[i],dim=0).tolist()))
   
   n_a = torch.FloatTensor(n_a).to(device)

   target_value = torch.min(target_q_net(next_state, n_a), target_q_net_2(next_state, n_a))
   next_q_value = reward + (1 - done) * gamma * target_value
   
   q_value_loss = criterion(expected_q_value, next_q_value.detach())

   if update_ratio % 1000 == 0:

     print("Value error = " + str(abs(next_q_value[0].detach() - expected_q_value[0])))
   
   q_value_loss_2 = criterion(expected_q_value_2, next_q_value.detach())

   soft_q_optimizer.zero_grad()
   q_value_loss.backward()
   #nn.utils.clip_grad_norm_(soft_q_net.parameters(), max_norm=1.0, norm_type=2)
   soft_q_optimizer.step()
   
   soft_q_optimizer_2.zero_grad()
   q_value_loss_2.backward()
   #nn.utils.clip_grad_norm_(soft_q_net_2.parameters(), max_norm=1.0, norm_type=2)
   soft_q_optimizer_2.step()
   
   #new_action = policy_net.forward(state)

   if update_ratio % 2 == 0:
     
     new_action = policy_net.forward(state)
      
     for i in range(len(new_action)):
         
        new_action[i] = torch.nn.functional.softmax(new_action[i])
      
     n_a = []
     for i in range(len(new_action)):

      #if np.random.uniform(0.0, 1.0) < 0.10:

        #n_a.append(np.random.randint(3))
      
      #else:
        
       n_a.append(torch.argmax(new_action[i],dim=0).tolist())

     n_a = torch.FloatTensor(n_a).to(device)

     expected_new_q_value = soft_q_net(state, n_a)  
     expected_new_q_value_2 = soft_q_net_2(state, n_a)
      
     log_prob_target = torch.min(expected_new_q_value, expected_new_q_value_2)

     policy_loss = -log_prob_target

     policy_loss = policy_loss.mean()

     policy_optimizer.zero_grad()
     policy_loss.sum().backward()
     #nn.utils.clip_grad_norm_(policy_net.parameters(), max_norm=1.0, norm_type=2)
     policy_optimizer.step()

     #if update_ratio % 1000 == 0:
     #print("TARGET UPDATE")
     soft_tau = 5e-3
     for target_param, param in zip(target_q_net.parameters(), soft_q_net.parameters()):
       target_param.data.copy_(target_param.data * (1.0 - soft_tau) + param.data * soft_tau)

     for target_param, param in zip(target_q_net_2.parameters(), soft_q_net_2.parameters()):
       target_param.data.copy_(target_param.data * (1.0 - soft_tau) + param.data * soft_tau)

   #Store losses for plot
   #if update_ratio % 200 == 0:
   #  total_q_value_loss.append(q_value_loss.item())
   #  total_q_value_loss_2.append(q_value_loss_2.item())
   #  total_policy_loss.append(policy_loss.sum())
   
    # PRINT LOSSES
   if (update_ratio % 100 == 0 and update == True):
     print('Iter :- ', update_ratio,
            ' Q_value Loss :- ', round(q_value_loss.data.item(), 2),
            ' Q_value Loss_2 :- ', round(q_value_loss_2.data.item(), 2),
            #' Value Loss :- ', round(value_loss.data.item(), 2),
            ' Policy Loss :- ', round(float(policy_loss.sum()),2))

In [22]:
# Initialize buffer with size of 1.10^6
replay_buffer_size = [100000, 10000]
replay_buffer = Bio_inspired_Buffer(replay_buffer_size)

soft_q_net = Critic(2, 3).to(device)
target_q_net = Critic(2, 3).to(device)

soft_q_net_2 = Critic(2, 3).to(device)
target_q_net_2 = Critic(2, 3).to(device)

policy_net = Actor(2, 3).to(device)

policy_net_perturbed = Actor(2, 3).to(device)

for target_param, param in zip(policy_net_perturbed.parameters(), policy_net.parameters()):
      target_param.data.copy_(param.data)

for target_param, param in zip(target_q_net.parameters(), soft_q_net.parameters()):
      target_param.data.copy_(param.data)

for target_param, param in zip(target_q_net_2.parameters(), soft_q_net_2.parameters()):
   target_param.data.copy_(param.data)

criterion = nn.SmoothL1Loss()

soft_q_optimizer = optim.Adam(soft_q_net.parameters(), lr=3e-4)#, weight_decay=wd)
soft_q_optimizer_2 = optim.Adam(soft_q_net_2.parameters(), lr=3e-4)#, weight_decay=wd)
policy_optimizer = optim.Adam(policy_net.parameters(), lr=3e-4) #, weight_decay=wd)

env = gym.make('MountainCar-v0')

obs_space = env.observation_space
action_space = env.action_space
print("The observation space: {}".format(obs_space))
print("The action space: {}".format(action_space))

# Number of steps you run the agent for 
num_steps = 200
update_ratio = 0

noise_std = 2.0

epsilon = 1.0

nb_step = 0

cpt = False

reward_ep = -200

The observation space: Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
The action space: Discrete(3)


In [23]:
replay_buffer = Bio_inspired_Buffer(replay_buffer_size)

epsilon = 1.0

nb_success = 0
list_success = []

for ep in range(3000):

   state = [0.0] * 10

   if cpt == True:

      i = 0

      while i < 200:

         soft_q_update(1, update_ratio, update=True)
         update_ratio += 1
         i += 1
   '''
   if cpt == True:
        
      for target_param, param in zip(policy_net_perturbed.parameters(), policy_net.parameters()):
          target_param.data.copy_(param.data)

      with torch.no_grad():

         policy_net_perturbed.noise(noise_std)
         #soft_q_net.noise(noise_std)
         #soft_q_net_2.noise(noise_std)
      
      noise_std = noise_std/1.005

      print("Noise Std = " + str(noise_std))
   '''
   obs = env.reset()
   '''
   for i in obs:
      list_obs.pop(0)
      list_obs.append(i)
   obs = list_obs
   print("Episode : " +str(ep))
   '''

   print("Episode : " +str(ep))

   #obs = list_obs

   reward_ep = 0

   if cpt == False:

      policy_net = Actor(2, 3).to(device)
   
   for i in obs:
     state.pop(0)
     state.append(i)
  
   if ep % 100 == 0:

     list_success.append(nb_success)
     nb_success = 0
     print("//////")
     print("Total success every 100 episodes: " + str(list_success))
     print("//////")

   for step in range(num_steps):
       # take random action, but you can also do something more intelligent
       # action = my_intelligent_agent_fn(obs) 

       nb_step += 1
             
       #else:
       
       with torch.no_grad():

          #if cpt == False:

            #action_apply = policy_net_perturbed.forward(torch.FloatTensor(state).to(device)) #env.action_space.sample()

          #else:

          action_apply = policy_net.forward(torch.FloatTensor(state).to(device))

          action_apply = torch.nn.functional.softmax(action_apply)

          action_apply = torch.argmax(action_apply,dim=0).tolist()
       
       if np.random.uniform(0.0, 1.0) < 0.01 and cpt == True:
         
         action_apply = np.random.randint(3)
       '''
       if epsilon > 0.10:

         epsilon = epsilon - 0.000001

       else:

         epsilon = 0.10

       if nb_step % 10000 == 0:
         print("/////")
         print("Epsilon  = " + str(epsilon))
         print("/////")
       '''
       # apply the action
       next_state, reward, done, info = env.step(action_apply)
       '''
       for i in next_state:
         
         list_obs.pop(0)
         list_obs.append(i)
       next_state = list_obs
       '''
       next_s = state
       
       for i in next_state:
         next_s.pop(0)
         next_s.append(i)
       
       replay_buffer.push(np.float32(state), float(action_apply), reward, np.float32(next_s), done, 0)
       '''
       for i in range(len(obs)):
         
         list_obs.pop(0)
         list_obs.append(obs[i]- next_state[i])

       for i in range(len(list_obs)):

         list_obs[i] = list_obs[i] + np.random.normal(0.0, 0.01)
       '''
       
       #obs = next_state
       state = next_s

       reward_ep += reward

       # Render the env
       #env.render()

       # Wait a bit before the next frame unless you want to see a crazy fast video
       time.sleep(0.001)
       
       if step % 100 == 0:

         print("Step number = " + str(step) + " Reward = " + str(reward))
       
       # If the epsiode is up, then start another one
       if done:

         print("Total reward episode = " + str(reward_ep))

         if reward_ep > -200:

            nb_success += 1
            cpt = True
            print("////////")
            print("Goal Reached !!")
            print("////////")
         
         #if cpt == False:

           #replay_buffer = Bio_inspired_Buffer(replay_buffer_size)
         
         env.reset()

   # Close the env
   #env.close()

Episode : 0
//////
Total success every 100 episodes: [0]
//////
Step number = 0 Reward = -1.0
Step number = 100 Reward = -1.0


<ipython-input-23-ab059c2c6cfa>:86: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  action_apply = torch.nn.functional.softmax(action_apply)


Total reward episode = -200.0
Episode : 1
Step number = 0 Reward = -1.0
Step number = 100 Reward = -1.0
Total reward episode = -200.0
Episode : 2
Step number = 0 Reward = -1.0
Step number = 100 Reward = -1.0
Total reward episode = -200.0
Episode : 3
Step number = 0 Reward = -1.0
Step number = 100 Reward = -1.0
Total reward episode = -200.0
Episode : 4
Step number = 0 Reward = -1.0
Step number = 100 Reward = -1.0
Total reward episode = -200.0
Episode : 5
Step number = 0 Reward = -1.0
Step number = 100 Reward = -1.0
Total reward episode = -200.0
Episode : 6
Step number = 0 Reward = -1.0
Step number = 100 Reward = -1.0
Total reward episode = -200.0
Episode : 7
Step number = 0 Reward = -1.0
Step number = 100 Reward = -1.0
Total reward episode = -200.0
Episode : 8
Step number = 0 Reward = -1.0
Step number = 100 Reward = -1.0
Total reward episode = -200.0
Episode : 9
Step number = 0 Reward = -1.0
Step number = 100 Reward = -1.0
Total reward episode = -200.0
Episode : 10
Step number = 0 Rewar

<ipython-input-18-e144b223de71>:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  new_action[i] = torch.nn.functional.softmax(new_action[i]) #+np.random.normal(0.0, 0.1))
<ipython-input-18-e144b223de71>:79: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  new_action[i] = torch.nn.functional.softmax(new_action[i])


Iter :-  100  Q_value Loss :-  0.03  Q_value Loss_2 :-  0.33  Policy Loss :-  0.35
Episode : 220
Step number = 0 Reward = -1.0
Step number = 100 Reward = -1.0
Total reward episode = -166.0
////////
Goal Reached !!
////////
Iter :-  200  Q_value Loss :-  0.16  Q_value Loss_2 :-  0.8  Policy Loss :-  1.02
Iter :-  300  Q_value Loss :-  0.21  Q_value Loss_2 :-  0.3  Policy Loss :-  1.14
Episode : 221
Step number = 0 Reward = -1.0
Step number = 100 Reward = -1.0
Total reward episode = -168.0
////////
Goal Reached !!
////////
Iter :-  400  Q_value Loss :-  0.0  Q_value Loss_2 :-  0.0  Policy Loss :-  1.24
Iter :-  500  Q_value Loss :-  0.0  Q_value Loss_2 :-  0.0  Policy Loss :-  1.63
Episode : 222
Step number = 0 Reward = -1.0
Step number = 100 Reward = -1.0
Total reward episode = -173.0
////////
Goal Reached !!
////////
Iter :-  600  Q_value Loss :-  0.0  Q_value Loss_2 :-  0.0  Policy Loss :-  1.74
Iter :-  700  Q_value Loss :-  0.04  Q_value Loss_2 :-  0.07  Policy Loss :-  1.97
Episode

KeyboardInterrupt: ignored